In [47]:
from utils import read_lines
from collections import defaultdict
from queue import SimpleQueue
from threading import Thread

def parse_input(input_file):
    return [line.split(' ') for line in read_lines(input_file)]

def get_value(a, reg):
    try:
        return int(a)
    except ValueError:
        return reg[a]

def part1(input_file):
    instructions = parse_input(input_file)
    ans = 0
    reg = defaultdict(int)
    i = 0
    while 0 <= i < len(instructions):
        ins = instructions[i]
        match ins[0]:
            case 'snd':
                ans = get_value(ins[1], reg)
                i += 1
            case 'set':
                reg[ins[1]] = get_value(ins[2], reg)
                i += 1
            case 'add':
                reg[ins[1]] += get_value(ins[2], reg)
                i += 1
            case 'mul':
                reg[ins[1]] *= get_value(ins[2], reg)
                i += 1
            case 'mod':
                reg[ins[1]] %= get_value(ins[2], reg)
                i += 1
            case 'rcv':
                if get_value(ins[1], reg):
                    return ans
                i += 1
            case 'jgz':
                if get_value(ins[1], reg) > 0:
                    i += get_value(ins[2], reg)
                else:
                    i += 1
            case _:
                raise ValueError(f'unkonwn instruction {ins}')
            

class Duet:
    def __init__(self, instructions):
        self.instructions = instructions
        self.blocking0 = False
        self.blocking1 = False
        self.send_count = 0

    def start(self):
        q0, q1 = SimpleQueue(), SimpleQueue()
        t0 = Thread(target=self.run_program, args=(0, q1, q0))
        t1 = Thread(target=self.run_program, args=(1, q0, q1))
        t0.start()
        t1.start()
        t0.join()
        t1.join()
        return self.send_count
               
    def run_program(self, id, q_snd: SimpleQueue, q_rcv: SimpleQueue):
        reg = defaultdict(int)
        reg['p'] = id
        i = 0
        while 0 <= i < len(self.instructions):
            ins = self.instructions[i]
            match ins[0]:
                case 'snd':
                    if id == 1:
                        self.send_count += 1
                        self.blocking0 = False
                    else:
                        self.blocking1 = False
                    q_snd.put(get_value(ins[1], reg))
                    i += 1
                case 'set':
                    reg[ins[1]] = get_value(ins[2], reg)
                    i += 1
                case 'add':
                    reg[ins[1]] += get_value(ins[2], reg)
                    i += 1
                case 'mul':
                    reg[ins[1]] *= get_value(ins[2], reg)
                    i += 1
                case 'mod':
                    reg[ins[1]] %= get_value(ins[2], reg)
                    i += 1
                case 'rcv':
                    if q_rcv.empty():
                        if id == 0:
                            self.blocking0 = True
                        else:
                            self.blocking1 = True
                        if self.blocking0 and self.blocking1:
                            q_snd.put('stop')
                            return
                    v = q_rcv.get(timeout=1)
                    if v == 'stop':
                        return
                    reg[ins[1]] = v
                    i += 1
                case 'jgz':
                    if get_value(ins[1], reg) > 0:
                        i += get_value(ins[2], reg)
                    else:
                        i += 1
                case _:
                    raise ValueError(f'unkonwn instruction {ins}')

def part2(input_file):
    instructions = parse_input(input_file)
    duet = Duet(instructions)
    return duet.start()

In [48]:
part1('inputs/day18_test.txt')

4

In [49]:
part1('inputs/day18.txt')

8600

In [50]:
part2('inputs/day18.txt')

7239